In [2]:
import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars
import os
os.getcwd()

/u/zliu/datastor1/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/datastor1/zliu/mend/notebooks'

In [21]:
task = "musique"
edit_loss="clm"
edit_input="question"
prompt = "urial"
exp_name = "llama3.2-1B_on_musique" # llama3.2-1B_on_musiqueQonly
fpath = f"../exp_output/{exp_name}/{task}/mend_eval_loss={edit_loss}_input={edit_input}_n=1000_prompt={prompt}.xlsx"
# print("File name:", fpath)
df = pd.read_excel("/u/zliu/datastor1/mend/exp_output/llama3.2-1B_on_musiqueQonly/musique/mend_eval_loss=clm_input=question_n=1000_prompt=no_wo-gen.xlsx")
# macro_averaging(df, multi_level_averaging=["stage", "question"], metrics=['[Q][A] acc', "rouge1", "llm_accuracy"])
df.head()

,input,stage,predicted_answer_idx,[Q][A] Acc EM,[Q][A] Acc PM,[A]|[Q] Acc EM,[A]|[Q] Acc PM
0,Who is the spouse of the Green performer? Miqu...,pre-edit,0,0,0.428571,0,0.500000
1,Who is the spouse of the Green performer? Miqu...,post-edit,0,0,0.571429,0,0.833333
2,Who founded the company that distributed the f...,pre-edit,0,0,0.357143,0,0.500000
3,Who founded the company that distributed the f...,post-edit,0,0,0.785714,1,1.000000
4,What administrative territorial entity is the ...,pre-edit,0,0,0.444444,0,0.600000


In [22]:
macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM",])

,stage,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
0,post-edit,0.776777,0.936339,0.0,0.696579
1,pre-edit,0.000000,0.312354,0.0,0.308479


In [19]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [45]:
df = pd.read_excel("/u/zliu/datastor1/KE-by-CP/exp_output/musique_single_dev_Llama-3.2-1B/rag=oracle_0icl_tag=paragraph/all_results.xlsx")
df = df[df["id"].isin(dev_ids)]
print(len(df))
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["rouge1", "llm_accuracy"])

1000


,question_type,rouge1,llm_accuracy
0,multi_hop_efficacy,0.131099,0.3458


In [46]:
df = pd.read_excel("/u/zliu/datastor1/KE-by-CP/exp_output/musique_dev_Llama-3.2-1B/rag=oracle_0icl_tag=paragraph/all_results.xlsx")
df = df[df["id"].isin(dev_ids)]
print(len(df))
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["rouge1", "llm_accuracy"])

1000


,question_type,rouge1,llm_accuracy
0,multi_hop_efficacy,0.169209,0.4031


In [40]:
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["rouge1", "llm_accuracy"])

,question_type,rouge1,llm_accuracy
0,multi_hop_efficacy,0.060712,0.1399


In [15]:
df.insert(df.shape[-1], "haha", 9)

In [16]:
df

,predicted_answer_idx,haha
0,0,9
